教程地址：https://medium.com/data-science/object-based-land-cover-classification-with-python-cbe54e9c9e24  
对应视频地址：https://www.youtube.com/playlist?list=PLzHdTn7Pdxs6R6gf-0aLCqy8pL1GazPEe    
形状，纹理：
https://blog.csdn.net/qq_31988139/article/details/133910317  
https://patentimages.storage.googleapis.com/7a/86/7e/32c755861ca900/CN101930547A.pdf  
http://ch.whu.edu.cn/cn/article/pdf/preview/2432.pdf  

In [1]:
import numpy as np
from osgeo import gdal
from skimage import exposure
import time

naip_fn = r'D:\Projects\VsCode\Python\img_processing_system\qgis_image\naip\m_4211161_se_12_1_20160624\m_4211161_se_12_1_20160624.tif'

driverTiff = gdal.GetDriverByName('GTiff')
naip_ds = gdal.Open(naip_fn)
if naip_ds is None:
    print("naip_ds is None")
    del naip_ds
    exit()
nbands = naip_ds.RasterCount
band_data = []
for i in range(1, nbands+1):
    band = naip_ds.GetRasterBand(i).ReadAsArray()
    band_data.append(band)
band_data = np.dstack(band_data)
print(band_data.shape)

(7586, 5834, 4)


In [4]:
from skimage.segmentation import slic
# scale image values from 0.0 - 1.0
img = exposure.rescale_intensity(band_data)
npixels,x, nbands = img.shape
print(npixels,x, nbands)

7586 5834 4


In [ ]:
from skimage.segmentation import slic
# scale image values from 0.0 - 1.0
img = exposure.rescale_intensity(band_data)
print(img.shape)
# do segmentation multiple options with quickshift and slic
seg_start = time.time()
# segments = quickshift(img, convert2lab=False)
# segments = quickshift(img, ratio=0.8, convert2lab=False)
# segments = quickshift(img, ratio=0.99, max_dist=5, convert2lab=False)
# segments = slic(img, n_segments=100000, compactness=0.1)
# segments = slic(img, n_segments=500000, compactness=0.01)
segments = slic(img, n_segments=500000, compactness=0.1)
print('segments complete', time.time() - seg_start)
print(segments.shape)
print(img.shape)
# save segments to raster
segments_fn = r'D:\Projects\VsCode\Python\img_processing_system\qgis_image\naip\segments_final.tif'
segments_ds = driverTiff.Create(segments_fn, naip_ds.RasterXSize, naip_ds.RasterYSize,
                                1, gdal.GDT_Float32)
segments_ds.SetGeoTransform(naip_ds.GetGeoTransform())
segments_ds.SetProjection(naip_ds.GetProjectionRef())
segments_ds.GetRasterBand(1).WriteArray(segments)
segments_ds = None
del naip_ds

In [ ]:
import scipy
# takes all the pixels in a segment and calculates statistics(min, max, mean, variance, skewness, and kurtosis) 
# for each band, saving them in the features variable
def segment_features(segment_pixels):
    features = []
    npixels, nbands = segment_pixels.shape
    for b in range(nbands):
        stats = scipy.stats.describe(segment_pixels[:, b])
        band_stats = list(stats.minmax) + list(stats)[2:]
        if npixels == 1:
            # in this case the variance = nan, change it 0.0
            band_stats[3] = 0.0
        features += band_stats
    return features

print(segments)
segment_ids = np.unique(segments) # segments有重复值是什么意思？segments中的值即每个像素属于哪个segment，图片中若干像素属于同一个segment
objects = []
object_ids = []
print(segment_ids)
print("换行")
for id in segment_ids: # 这段代码在Intlel i7 8565U上需要运行6个小时。可以通过并行运行优化。我的电脑运行：使用了并行优化，跑了1h都没跑完。
    segment_pixels = img[segments == id] # segments中的值就是一个个segment_id。将图片中都属于同一个segment_id的像素值存在一起
    object_features = segment_features(segment_pixels)
    objects.append(object_features) # 每个segment就是一个object
    object_ids.append(id)


# 分割线

In [1]:
import numpy as np
from osgeo import gdal
from skimage import exposure
import time
import scipy
import concurrent.futures

# takes all the pixels in a segment and calculates statistics(min, max, mean, variance, skewness, and kurtosis) 
# for each band, saving them in the features variable
def segment_features(segment_pixels):
    features = []
    npixels, nbands = segment_pixels.shape
    for b in range(nbands):
        stats = scipy.stats.describe(segment_pixels[:, b])
        band_stats = list(stats.minmax) + list(stats)[2:]
        if npixels == 1:
            # in this case the variance = nan, change it 0.0
            band_stats[3] = 0.0
        features += band_stats
    return features


# Define a helper function to process one segment
def process_segment(segment_id):
    segment_pixels = img[segments == segment_id]
    object_features = segment_features(segment_pixels)
    return (segment_id, object_features)


if __name__ == '__main__':
    naip_fn = r'D:\Projects\VsCode\Python\img_processing_system\qgis_image\naip\m_4211161_se_12_1_20160624\m_4211161_se_12_1_20160624.tif'

    driverTiff = gdal.GetDriverByName('GTiff')
    naip_ds = gdal.Open(naip_fn)
    if naip_ds is None:
        print("naip_ds is None")
        del naip_ds
        exit()
    nbands = naip_ds.RasterCount
    band_data = []
    for i in range(1, nbands+1):
        band = naip_ds.GetRasterBand(i).ReadAsArray()
        band_data.append(band)
    band_data = np.dstack(band_data)
    from skimage.segmentation import slic
    # scale image values from 0.0 - 1.0
    img = exposure.rescale_intensity(band_data)

    # do segmentation multiple options with quickshift and slic
    seg_start = time.time()
    # segments = quickshift(img, convert2lab=False)
    # segments = quickshift(img, ratio=0.8, convert2lab=False)
    # segments = quickshift(img, ratio=0.99, max_dist=5, convert2lab=False)
    # segments = slic(img, n_segments=100000, compactness=0.1)
    # segments = slic(img, n_segments=500000, compactness=0.01)
    segments = slic(img, n_segments=500000, compactness=0.1)
    # save segments to raster
    segments_fn = r'D:\Projects\VsCode\Python\img_processing_system\qgis_image\naip\segments_final.tif'
    segments_ds = driverTiff.Create(segments_fn, naip_ds.RasterXSize, naip_ds.RasterYSize,
                                    1, gdal.GDT_Float32)
    segments_ds.SetGeoTransform(naip_ds.GetGeoTransform())
    segments_ds.SetProjection(naip_ds.GetProjectionRef())
    segments_ds.GetRasterBand(1).WriteArray(segments)
    segments_ds = None
    del naip_ds
    segment_ids = np.unique(segments) # segments有重复值是什么意思？segments中的值即每个像素属于哪个segment，图片中若干像素属于同一个segment
    objects = []
    object_ids = []
    # Use multiple CPU cores
    with concurrent.futures.ProcessPoolExecutor() as executor:
        results = list(executor.map(process_segment, segment_ids))

    # Unpack results
    object_ids, objects = zip(*results)


Traceback (most recent call last):
  File "d:\Projects\VsCode\Python\img_processing_system\.conda\Lib\multiprocessing\queues.py", line 246, in _feed
    send_bytes(obj)
  File "d:\Projects\VsCode\Python\img_processing_system\.conda\Lib\multiprocessing\connection.py", line 184, in send_bytes
    self._check_closed()
  File "d:\Projects\VsCode\Python\img_processing_system\.conda\Lib\multiprocessing\connection.py", line 137, in _check_closed
    raise OSError("handle is closed")
OSError: handle is closed
Traceback (most recent call last):
  File "d:\Projects\VsCode\Python\img_processing_system\.conda\Lib\multiprocessing\queues.py", line 246, in _feed
    send_bytes(obj)
  File "d:\Projects\VsCode\Python\img_processing_system\.conda\Lib\multiprocessing\connection.py", line 184, in send_bytes
    self._check_closed()
  File "d:\Projects\VsCode\Python\img_processing_system\.conda\Lib\multiprocessing\connection.py", line 137, in _check_closed
    raise OSError("handle is closed")
OSError: han

OSError: handle is closed